In [ ]:
## Lets import libraries we will use to prepare an input image.
import numpy as np
from matplotlib import pyplot as plt
import torch
%matplotlib inline
## From our examples we can import utility functions for inference:
from dle.inference import load_image, rescale, crop_center, normalize
from examples.SSD300_inference import load_checkpoint, build_predictor
from apex.fp16_utils import network_to_half
from ssd.utils import dboxes300_coco, Encoder
import matplotlib.patches as patches
import json
import torch

In [ ]:
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

In [ ]:
ssd_model.to('cpu')
ssd_model.eval()


In [ ]:
uris = [f'../img/jetbot/frame/frame_{i}.png' for i in range(87)]
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs)
tensor = tensor[:,:3,:,:].to('cpu')

In [ ]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]
classes_to_labels = utils.get_coco_object_dictionary()


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches

for image_idx in range(len(best_results_per_input)):
    fig, ax = plt.subplots(1)
    # Show original, denormalized image...
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    # ...with detections
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        left, bot, right, top = bboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
    plt.axis('off')
    plt.savefig(f'../img/jetbot/pred/pred_{image_idx}.png')
    plt.show()
